In [2]:
import pandas as pd
import numpy as np

In [3]:
checklist = pd.read_csv('/home/joon/data/checklist_taxonomy_preprocessed.csv')
checklist_genus_values = checklist.genus.unique()
checklist = checklist[['genus','accepted_plant_name_id','plant_name_id','first_published','lifeform_description','climate_description']]
accepted_plant_name_ids = checklist.accepted_plant_name_id.unique()
checklist.set_index('accepted_plant_name_id',inplace=True)

# Publication year, min, max, delta

In [739]:
def checklist_year_max(checklist):
    return np.max(checklist.first_published)

In [740]:
def checklist_year_min(checklist):
    return np.min(checklist.first_published)

In [741]:
def checklist_year_delta(checklist):
    return checklist_year_max(checklist)-checklist_year_min(checklist)

# Number of synonyms

In [742]:
def checklist_n_synonyms(checklist):
    return checklist.shape[0] if type(checklist)== pd.core.frame.DataFrame else 1

# Lifeform and climate description

In [743]:
def checklist_lifeform(checklist):
    return checklist.lifeform_description.mode(dropna=False)[0] if type(checklist)== pd.core.frame.DataFrame else checklist.lifeform_description

In [744]:
def checklist_climate(checklist):
    return checklist.climate_description.mode(dropna=False)[0] if type(checklist)==pd.core.frame.DataFrame else checklist.climate_description

In [745]:
lifeform_thomas = pd.read_csv('/home/joon/data/lifeform_thomas.csv',index_col=0,na_values='?')
nb_lifeform_thomas = lifeform_thomas.columns.size

In [746]:
# sanity check
set(checklist.lifeform_description.unique()[1:]) == set(lifeform_thomas.index)

True

In [747]:
def checklist_lifeform_thomas(selected_rows):
    lifeforms = list(selected_rows.lifeform_description.dropna().unique()) if type(selected_rows) == pd.core.frame.DataFrame else [selected_rows.lifeform_description]
    if lifeforms == [] or lifeforms == [np.nan]:
        return [np.nan for i in range(nb_lifeform_thomas)]
    else:
        return list(lifeform_thomas.loc[lifeforms[0]])

In [748]:
climate_thomas = pd.read_csv('/home/joon/data/climate_thomas.csv',index_col=0)
nb_climate_thomas = climate_thomas.columns.size

In [749]:
# sanity check
set(checklist.climate_description.dropna().unique()) == set(climate_thomas.index)

True

In [750]:
def checklist_climate_thomas(selected_rows):
    climates = list(selected_rows.climate_description.dropna().unique()) if type(selected_rows) == pd.core.frame.DataFrame else [selected_rows.climate_description]
    if climates == [] or climates == [np.nan]:
        return [np.nan for i in range(nb_climate_thomas)]
    else:
        return list(np.max(np.array([climate_thomas.loc[climate] for climate in climates]),axis=0))

In [778]:
single_column_functions = [checklist_year_max,checklist_year_min,checklist_year_delta,checklist_n_synonyms,checklist_lifeform,checklist_climate]
column_names = ['accepted_plant_name_id','checklist_year_max','checklist_year_min','checklist_year_delta','checklist_n_synonyms','checklist_lifeform','checklist_climate']+['lifeform_thomas_'+lifeform for lifeform in lifeform_thomas.columns]+['climate_thomas_'+climate for climate in climate_thomas.columns]

Pour chaque checklist accepted_plant_name_id

In [809]:
def checklist_taxonomy_based_features_compute(selected_rows):
    return [f(selected_rows) for f in single_column_functions]+checklist_lifeform_thomas(selected_rows)+checklist_climate_thomas(selected_rows)

In [849]:
def checklist_taxonomy_based_features_process_genus(genus):
    df = pd.DataFrame([])

    checklist_from_genus = checklist[checklist.genus==genus]
    checklist_from_genus = checklist_from_genus[checklist_from_genus.index == checklist_from_genus.plant_name_id]
    for accepted_plant_name_id in checklist_from_genus.index.unique():
        selected_rows = checklist.loc[accepted_plant_name_id]
        df = df.append(pd.Series([accepted_plant_name_id]+checklist_taxonomy_based_features_compute(selected_rows)),ignore_index=True)
    if not df.empty:
        df.to_csv('/home/joon/data/checklist-taxonomy-based-features-'+genus+'.csv',header=False,index=False)

In [855]:
os.system('find /home/joon/data -type f -name \'checklist-taxonomy-based-features*.csv\' | xargs rm')

from multiprocessing import Pool
with Pool(processes=11) as pool:
    pool.map(checklist_taxonomy_based_features_process_genus, checklist_genus_values)

pd.DataFrame([column_names]).to_csv('/home/joon/data/checklist-taxonomy-based-features-column-names.csv',header=None,index=None)

#fusionner les résultats et supprimer les fichiers temporaires!
os.system('find /home/joon/data -type f -name \'checklist-taxonomy-based-features-*.csv\' | xargs cat /home/joon/data/checklist-taxonomy-based-features-column-names.csv > /home/joon/data/checklist-taxonomy-based-features.csv')
os.system('find /home/joon/data -type f -name \'checklist-taxonomy-based-features-*.csv\' | xargs rm')

/usr/lib/python3.7/site-packages/pandas/core/algorithms.py:881: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'float'
  warn("Unable to sort modes: {error}".format(error=e))


/usr/lib/python3.7/site-packages/pandas/core/algorithms.py:881: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'float'
  warn("Unable to sort modes: {error}".format(error=e))


/usr/lib/python3.7/site-packages/pandas/core/algorithms.py:881: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'float'
  warn("Unable to sort modes: {error}".format(error=e))


/usr/lib/python3.7/site-packages/pandas/core/algorithms.py:881: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'float'
  warn("Unable to sort modes: {error}".format(error=e))


/usr/lib/python3.7/site-packages/pandas/core/algorithms.py:881: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'float'
  warn("Unable to sort modes: {error}".format(error=e))


...

/usr/lib/python3.7/site-packages/pandas/core/algorithms.py:881: UserWarning: Unable to sort modes: '<' not supported between instances of 'float' and 'str'
  warn("Unable to sort modes: {error}".format(error=e))


/usr/lib/python3.7/site-packages/pandas/core/algorithms.py:881: UserWarning: Unable to sort modes: '<' not supported between instances of 'float' and 'str'
  warn("Unable to sort modes: {error}".format(error=e))


/usr/lib/python3.7/site-packages/pandas/core/algorithms.py:881: UserWarning: Unable to sort modes: '<' not supported between instances of 'float' and 'str'
  warn("Unable to sort modes: {error}".format(error=e))


/usr/lib/python3.7/site-packages/pandas/core/algorithms.py:881: UserWarning: Unable to sort modes: '<' not supported between instances of 'float' and 'str'
  warn("Unable to sort modes: {error}".format(error=e))


0

In [856]:
checklist_taxonomy_based_features = pd.read_csv('/home/joon/data/checklist-taxonomy-based-features.csv')
if np.array_equal(checklist_taxonomy_based_features.iloc[-1].values,checklist_taxonomy_based_features.columns.values):
    checklist_taxonomy_based_features.drop(index=checklist_taxonomy_based_features.iloc[-1].name,inplace=True)

MATCHING: CHECKLIST BASED FEATURES FOR IUCN

In [1050]:
def read_set_in_csv(x):
    if x=='set()':
        return set()
    else:
        return set(x.replace("'",'').strip('{}').split(', '))

In [1051]:
iucn_checklist_matching = pd.read_csv('/home/joon/data/iucn-checklist-matching.csv',index_col=0,header=None,squeeze=True,converters={1:read_set_in_csv})

In [1052]:
df = pd.DataFrame([])

for iucn_taxon_index in iucn_checklist_matching.index:
    selected_accepted_plant_name_ids = iucn_checklist_matching.loc[iucn_taxon_index]
    selected_rows = checklist.loc[selected_accepted_plant_name_ids]
    if selected_accepted_plant_name_ids != set():
        df = df.append(pd.Series([iucn_taxon_index]+checklist_taxonomy_based_features_compute(selected_rows)),ignore_index=True)
    else:
        df = df.append(pd.Series([iucn_taxon_index]+[np.nan for i in column_names[1:]]),ignore_index=True)

df.columns = ['iucn_taxon_index']+column_names[1:]
df.iucn_taxon_index = df.iucn_taxon_index.astype(int)
df.to_csv('/home/joon/data/checklist-taxonomy-based-features-for-iucn.csv',index=False)

/usr/lib/python3.7/site-packages/pandas/core/algorithms.py:881: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'float'
  warn("Unable to sort modes: {error}".format(error=e))


/usr/lib/python3.7/site-packages/pandas/core/algorithms.py:881: UserWarning: Unable to sort modes: '<' not supported between instances of 'float' and 'str'
  warn("Unable to sort modes: {error}".format(error=e))
